# Module

## Basic structures and features

This section is a direct copy of the topic on [modules](https://www.haskell.org/tutorial/modules.html) of [A Gentle Introduction to Haskell, Version 98](https://www.haskell.org/tutorial/index.html).

A Haskell program consists of a collection of modules. A module in Haskell serves the dual purpose of controlling name-spaces and creating abstract data types.

The top level of a module contains any of the various declarations we have discussed: fixity declarations, data and type declarations, class and instance declarations, type signatures, function definitions, and pattern bindings. Except for the fact that import declarations (to be described shortly) must appear first, the declarations may appear in any order (the top-level scope is mutually recursive).

Haskell's module design is relatively conservative: the name-space of modules is completely flat, and modules are in no way "first-class." Module names are alphanumeric and must begin with an uppercase letter. There is no formal connection between a Haskell module and the file system that would (typically) support it. In particular, there is no connection between module names and file names, and more than one module could conceivably reside in a single file (one module may even span several files). Of course, a particular implementation will most likely adopt conventions that make the connection between modules and files more stringent.

Technically speaking, a module is really just one big declaration which begins with the keyword module; here's an example for a module whose name is Tree:

In [1]:
module Tree ( Tree(Leaf,Branch), fringe ) where

data Tree a                = Leaf a | Branch (Tree a) (Tree a) 

fringe :: Tree a -> [a]
fringe (Leaf x)            = [x]
fringe (Branch left right) = fringe left ++ fringe right

This module explicitly exports `Tree`, `Leaf`, `Branch`, and `fringe`. If the export list following the module keyword is omitted, all of the names bound at the top level of the module would be exported. (In the above example everything is explicitly exported, so the effect would be the same.) Note that the name of a type and its constructors have be grouped together, as in `Tree(Leaf,Branch)`. As short-hand, we could also write `Tree(..)`. Exporting a subset of the constructors is also possible. The names in an export list need not be local to the exporting module; any name in scope may be listed in an export list.

The Tree module may now be imported into some other module:

In [2]:
module Main (main) where
import Tree ( Tree(Leaf,Branch), fringe )

main = print (fringe (Branch (Leaf 1) (Leaf 2)))

In [3]:
main

[1,2]

The various items being imported into and exported out of a module are called entities. Note the explicit import list in the import declaration; omitting it would cause all entities exported from Tree to be imported.

### Qualified Names
There is an obvious problem with importing names directly into the namespace of module. What if two imported modules contain different entities with the same name? Haskell solves this problem using qualified names. An import declaration may use the qualified keyword to cause the imported names to be prefixed by the name of the module imported. These prefixes are followed by the `.` character without intervening whitespace. [Qualifiers are part of the lexical syntax. Thus, `A.x` and `A . x` are quite different: the first is a qualified name and the second a use of the infix `.` function.] For example, using the Tree module introduced above:

In [4]:
module Fringe(fringe) where
import Tree(Tree(..))

data Extensions a = Extensions{eLeft::[a], eRight::[a]}

fringe :: Tree a -> [a]   -- A different definition of fringe
fringe (Leaf x) = [x]
fringe (Branch x y) = fringe x

In [5]:
module Main where
import Tree ( Tree(Leaf,Branch), fringe )
import qualified Fringe ( fringe )  

main = do print (fringe (Branch (Leaf 1) (Leaf 2)))
          print (Fringe.fringe (Branch (Leaf 1) (Leaf 2)))

In [6]:
main

[1,2]
[1]

### Data abstraction
Aside from controlling namespaces, modules provide the only way to build abstract data types (ADTs) in Haskell. For example, the characteristic feature of an ADT is that the representation type is hidden; all operations on the ADT are done at an abstract level which does not depend on the representation. For example, although the Tree type is simple enough that we might not normally make it abstract, a suitable ADT for it might include the following operations:
```haskell
data Tree a             -- just the type name 
leaf                    :: a -> Tree a
branch                  :: Tree a -> Tree a -> Tree a
cell                    :: Tree a -> a
left, right             :: Tree a -> Tree a
isLeaf                  :: Tree a -> Bool
```
A module supporting this is:

In [7]:
module TreeADT
(Tree
,leaf
,branch
,cell
,left
,right
,isLeaf) where

data Tree a             = Leaf a | Branch (Tree a) (Tree a) 

leaf                    = Leaf
branch                  = Branch
cell  (Leaf a)          = a
left  (Branch l r)      = l
right (Branch l r)      = r
isLeaf   (Leaf _)       = True
isLeaf   _              = False

Note in the export list that the type name Tree appears alone (i.e. without its constructors). Thus Leaf and Branch are not exported, and the only way to build or take apart trees outside of the module is by using the various (abstract) operations. Of course, the advantage of this information hiding is that at a later time we could change the representation type without affecting users of the type.

### More features

Here is a brief overview of some other aspects of the module system. See the report for more details.

- An import declaration may selectively hide entities using a hiding clause in the import declaration. This is useful for explicitly excluding names that are used for other purposes without having to use qualifiers for other imported names from the module.
- An import may contain an as clause to specify a different qualifier than the name of the importing module. This can be used to shorten qualifiers from modules with long names or to easily adapt to a change in module name without changing all qualifiers.
- Programs implicitly import the Prelude module. An explicit import of the Prelude overrides the implicit import of all Prelude names. Thus,

    ```haskell
    import Prelude hiding length
    ```
    will not import length from the Standard Prelude, allowing the name length to be defined differently.
- Instance declarations are not explicitly named in import or export lists. Every module exports all of its instance declarations and every import brings all instance declarations into scope.
- Class methods may be named either in the manner of data constructors, in parentheses following the class name, or as ordinary variables.

Although Haskell's module system is relatively conservative, there are many rules concerning the import and export of values. Most of these are obvious---for instance, it is illegal to import two different entities having the same name into the same scope. Other rules are not so obvious---for example, for a given type and class, there cannot be more than one instance declaration for that combination of type and class anywhere in the program. The reader should read the Report for details ([§5](https://www.haskell.org/onlinereport/modules.html#modules)).

## Structure in the project level
This section is from the section [Making our own modules](http://learnyouahaskell.com/modules#making-our-own-modules) of the online book [Learn You a Haskell for Great Good](http://learnyouahaskell.com/chapters).

### Flat structure module

Let's look at a simple geometry example:

In [8]:
module Geometry  
( sphereVolume  
, sphereArea  
, cubeVolume  
, cubeArea  
, cuboidArea  
, cuboidVolume  
) where  
  
sphereVolume :: Float -> Float  
sphereVolume radius = (4.0 / 3.0) * pi * (radius ^ 3)  
  
sphereArea :: Float -> Float  
sphereArea radius = 4 * pi * (radius ^ 2)  
  
cubeVolume :: Float -> Float  
cubeVolume side = cuboidVolume side side side  
  
cubeArea :: Float -> Float  
cubeArea side = cuboidArea side side side  
  
cuboidVolume :: Float -> Float -> Float -> Float  
cuboidVolume a b c = rectangleArea a b * c  
  
cuboidArea :: Float -> Float -> Float -> Float  
cuboidArea a b c = rectangleArea a b * 2 + rectangleArea a c * 2 + rectangleArea c b * 2  
  
rectangleArea :: Float -> Float -> Float  
rectangleArea a b = a * b  

Because a cube is only a special case of a cuboid, we defined its area and volume by treating it as a cuboid whose sides are all of the same length. We also defined a helper function called `rectangleArea`, which calculates a rectangle's area based on the lenghts of its sides. It's rather trivial because it's just multiplication. Notice that we used it in our functions in the module (namely `cuboidArea` and `cuboidVolume`) but we didn't export it! Because we want our module to just present functions for dealing with three dimensional objects, we used `rectangleArea` but we didn't export it.

To use our module, we just do:

```haskell
import Geometry
```

`Geometry.hs` has to be in the same folder that the program that's importing it is in, though.

## Hierarchical structures

Modules can also be given a hierarchical structures. Each module can have a number of sub-modules and they can have sub-modules of their own. Let's section these functions off so that Geometry is a module that has three sub-modules, one for each type of object.

First, we'll make a folder called Geometry. Mind the capital G. In it, we'll place three files: `Sphere.hs`, `Cuboid.hs`, and `Cube.hs`. Here's what the files will contain:

`Sphere.hs`

In [9]:
module Geometry.Sphere  
( volume  
, area  
) where  
  
volume :: Float -> Float  
volume radius = (4.0 / 3.0) * pi * (radius ^ 3)  
  
area :: Float -> Float  
area radius = 4 * pi * (radius ^ 2)  

`Cuboid.hs`

In [10]:
module Geometry.Cuboid  
( volume  
, area  
) where  
  
volume :: Float -> Float -> Float -> Float  
volume a b c = rectangleArea a b * c  
  
area :: Float -> Float -> Float -> Float  
area a b c = rectangleArea a b * 2 + rectangleArea a c * 2 + rectangleArea c b * 2  
  
rectangleArea :: Float -> Float -> Float  
rectangleArea a b = a * b  

`Cube.hs`

In [11]:
module Geometry.Cube  
( volume  
, area  
) where  
  
import qualified Geometry.Cuboid as Cuboid  
  
volume :: Float -> Float  
volume side = Cuboid.volume side side side  
  
area :: Float -> Float  
area side = Cuboid.area side side side  

Notice how we placed it in a folder called `Geometry` and then defined the module name as `Geometry.Sphere`. We did the same for the `cuboid`. Also notice how in all three sub-modules, we defined functions with the same names. We can do this because they're separate modules. We want to use functions from `Geometry.Cuboid` in `Geometry.Cube` but we can't just straight up do import `Geometry.Cuboid` because it exports functions with the same names as `Geometry.Cube`. That's why we do a qualified import and all is well.

So now if we're in a file that's on the same level as the Geometry folder, we can do, say:

In [12]:
import Geometry.Sphere 

And then we can call area and volume and they'll give us the `area` and `volume` for a sphere. And if we want to juggle two or more of these modules, we have to do qualified imports because they export functions with the same names. So we just do something like:

In [13]:
import qualified Geometry.Sphere as Sphere  
import qualified Geometry.Cuboid as Cuboid  
import qualified Geometry.Cube as Cube

And then we can call `Sphere.area`, `Sphere.volume`, `Cuboid.area`, etc. and each will calculate the area or volume for their corresponding object.

The next time you find yourself writing a file that's really big and has a lot of functions, try to see which functions serve some common purpose and then see if you can put them in their own module. You'll be able to just import your module the next time you're writing a program that requires some of the same functionality.

## Managing project modules
Chapter 4 of the book {cite}`bragilevsky2021haskell` has very thourough describtion on this subject.

### Example: container.mini
The goal of this example is to present the process of library development and turning it into a fully fledged yet simple independent Haskell package. 

Example outline:
- https://github.com/bravit/containers-mini/tree/v1.0
- We can start development without thinking about packaging.
- We use import and export lists to implement abstract data types.
- Compiling to an executable is done via ghc.
- We run source code via runhaskell.

While developing a library, we want to make sure that all our functions pass tests. We
are also interested in benchmarking them. Let’s create a directory with the following
structure:

```bash
$ tree containers-mini
containers-mini
├── Bench.hs
├── containers-mini.cabal
├── Data
│   ├── Deque.hs
│   ├── Queue.hs
│   └── Stack.hs
├── package.yaml
├── Setup.hs
├── TestQueue.hs
└── TestStack.hs
```

The Data subdirectory contains our data structure modules: `Data.Stack`, `Data.Queue`, and `Data.Deque`. We have test programs and benchmarks outside it.

As for the `Stack.hs`, let's load it into GHCi as follows:

```bash
containers-mini $ ghci Data/Stack.hs
ghci> top $ push 10 empty
Just 10
```
Let’s write the following simple testing scenario in the `TestStack.hs` file:
```haskell
import Data.Stack
main = do
    let st = push 15 $ push 10 $ push 5 $ push 0 empty
    -- st == 0, 5, 10, 15 <<-- top
    st' = pop $ pop st -- 0, 5 <<-- top
    st'' = push 100 st' -- 0, 5, 100 <<-- top
    shouldBeTrue = [ top st' == Just 5,
    top st'' == Just 100,
    isEmpty $ pop $ pop st']
print $ and shouldBeTrue
```
To run the test we could compile the program as follows (in archlinux, one need the -dynamic flag to load the dynamic system library):
```bash
containers-mini $ ghc -dynamic TestStack.hs
[1 of 2] Compiling Data.Stack       ( Data/Stack.hs, Data/Stack.o )
[2 of 2] Compiling Main             ( TestStack.hs, TestStack.o )
Linking TestStack ...
```
To run the test:
```bash
containers-mini $ ./TestStack
True
```
Alternatively, we could use the `runhaskell` wrapper, as shown next:
```bash
containers-mini $ runhaskell TestStack.h
True
```

The rest of the package can be tested in the similar fashion. Notice in this module, we use  explicit `Prelude`, as shown in the head portion of the `Deque.hs`, because many `Data.Sequence` functions have the same names as list functions from Prelude. Alternatively, I could use `import Prelude ()` or name hiding.

```haskell
{-# LANGUAGE NoImplicitPrelude #-}

module Data.Deque (
  Deque,
  empty,
  isEmpty,
  front,
  back,
  push_back,
  push_front,
  pop_back,
  pop_front) where

import Data.Sequence hiding (empty)
import qualified Data.Sequence as Seq
import Data.Bool (Bool)
import Data.Maybe (Maybe(..))
....
```
Without `Prelude`, we have to import everything, including Bool and Maybe, with data constructors. Note that We’ve `imported Data.Sequence` twice to disambiguate empty.

Next we are going to install the package `timeit` which can be found on Hackage: http://hackage.haskell.org/package/timeit. With this package, we can implement benchmarking as follows:
```haskell
import Data.Deque as D
import Data.Stack as S
import Data.List
import System.TimeIt

sumAll s view remove = sum $ unfoldr iter s
  where
    iter s = view s >>= \x -> Just (x, remove s)

fill n insert s = foldl' (flip insert) s [1..n]

main = do
  let n = 10^6
  timeItNamed "Stack" $
    print $ sumAll (fill n push S.empty) top pop
  timeItNamed "Deque" $
    print $ sumAll (fill n push_front D.empty) front pop_front
```
This is save in the file `Bench.hs`. To install `timeit` manually, here are the steps that consists of downloading the package, unpacking it, configuring it, building it and installing it:
```bash
containers-mini $ curl http://hackage.haskell.org/package/timeit-2.0/timeit-2.0.tar.gz --output timeit-2.0.tar.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2117  100  2117    0     0   3528      0 --:--:-- --:--:-- --:--:--  3534
containers-mini $ tar -xf timeit-2.0.tar.gz
containers-mini $ cd timeit-2.0
timeit-2.0 $ runhaskell Setup.hs configure
Configuring timeit-2.0...
timeit-2.0 $ runhaskell Setup.hs build
Preprocessing library for timeit-2.0..
Building library for timeit-2.0..
[1 of 1] Compiling System.TimeIt    ( System/TimeIt.hs, dist/build/System/TimeIt.o, dist/build/System/TimeIt.dyn_o )
timeit-2.0 $ sudo runhaskell Setup.hs install
Registering library for timeit-2.0..
```
Once we are done with installation, we can get back to running our benchmark as follows:
```bash
containers-mini $ runhaskell Bench.hs
500000500000
Stack:   0.57s
500000500000
Deque:   0.69s
```
The result is as expected. Lists when operated with the head element only are more efficient than sequences used in the same style.


```{bibliography}
:filter: docname in docnames
```